In [2]:
from bs4 import BeautifulSoup

In [5]:
import time
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def duckduckgo_search(query, max_results=10, headless=False):
    options = uc.ChromeOptions()
    
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--remote-debugging-port=9222")

    driver = uc.Chrome(options=options, version_main=136)

    try:
        driver.get("https://duckduckgo.com/")

        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "searchbox_input"))
        )
        search_box.send_keys(query)
        search_box.send_keys(Keys.RETURN)


        time.sleep(3)  # give results a moment to fully load

        source=driver.page_source
        return BeautifulSoup(source,"html.parser")

    except Exception as e:
        print("Error during scraping:", e)
        return []

    finally:
        driver.quit()




In [6]:
def amazon_asin(query:str):
    query=str(query)
    query=query.replace('\r', '').replace('\n', '').replace('\t','')
    if query=='' or query=='nan':
        return None
    search_query="site:amazon.com "+str(query)
    page= duckduckgo_search(search_query,headless=False)
    links=[link.attrs['href'] for link in page.find_all('a',class_='eVNpHGjtxRBq_gLOfGDr LQNqh2U1kzYxREs65IJu')]
    asins=[]
    for link in links:
        index=link.find('/dp/')
        if index!=-1:
            asins.append(link[(link.find('/dp/')+4):])
    if asins==list():
        return None
    return asins[0]

In [7]:
import pandas as pd
reports=pd.read_csv("../Data/IncidentReports.csv",encoding='cp1252')
reports=reports.loc[reports['Product Category']=='Toys & Children']


/tmp/ipykernel_6414/2015962264.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  reports=pd.read_csv("../Data/IncidentReports.csv",encoding='cp1252')


In [8]:
reports.loc[reports['Brand'].isna() ,'Brand']=reports['Manufacturer / Importer / Private Labeler Name']

In [12]:
reports['Title']=reports['Brand']+' '+reports['Model Name or Number']

In [13]:
reports.loc[reports.Title.isna(),'Title']=reports['Product Description']

In [14]:
reports['Guess_ASIN']=reports['Title']

In [ ]:
reports['ASIN']=reports['Guess_ASIN'].apply(amazon_asin)

In [22]:
step=10
for i in range(0,3117,step):
    print(i,"of", 3117)
    reports.loc[reports.index[i:i+step],'ASIN']=reports.loc[reports.index[i:i+step],'Guess_ASIN'].apply(amazon_asin)


0 of 3117
10 of 3117
20 of 3117
30 of 3117
40 of 3117
50 of 3117
60 of 3117
70 of 3117
80 of 3117
90 of 3117
100 of 3117
110 of 3117
120 of 3117
130 of 3117
140 of 3117
150 of 3117
160 of 3117
170 of 3117
180 of 3117
190 of 3117
200 of 3117
210 of 3117
220 of 3117
230 of 3117
240 of 3117
250 of 3117
260 of 3117
270 of 3117
280 of 3117
290 of 3117
300 of 3117
310 of 3117
320 of 3117
330 of 3117
340 of 3117
350 of 3117
360 of 3117
370 of 3117
380 of 3117
390 of 3117
400 of 3117
410 of 3117
420 of 3117
430 of 3117
440 of 3117
450 of 3117
460 of 3117
470 of 3117
480 of 3117
490 of 3117
500 of 3117
510 of 3117
520 of 3117
530 of 3117
540 of 3117
550 of 3117
560 of 3117
570 of 3117
580 of 3117
590 of 3117
600 of 3117
610 of 3117
620 of 3117
630 of 3117
640 of 3117
650 of 3117
660 of 3117
670 of 3117
680 of 3117
690 of 3117
700 of 3117
710 of 3117
720 of 3117
730 of 3117
740 of 3117
750 of 3117
760 of 3117
770 of 3117
780 of 3117
790 of 3117
800 of 3117
810 of 3117
820 of 3117
830 of 3117
840

In [23]:
reports[['Guess_ASIN','ASIN']]

,Guess_ASIN,ASIN
5,Glow & Save Sleeping Otter,B07N1JP56L
18,Green Elephant Home and Toys Pastel/Macaron co...,B084GJBJSC
29,Anbalulu AN-100,B09XMN7YGW
35,"Walmart Way to Celebrate Whale- EGT65209B, Di...",B08P4VG51K
47,Bunnypal Same,B0DZ2KN7RJ
...,...,...
61891,Circo Unsure,B0F231YH5Q
61908,Recalled Graco Stroller (the one with fingerti...,None
61939,Yookidoo D40117,B09TKTSWNT
61943,Airzone Pop stick,B005QNYGYC


In [24]:
reports.to_csv("asin_guess.csv")